# Base de dades pel menjar


In [1]:
import pandas as pd
import numpy as np
from io import StringIO

def ordenar_llista(llista):
    ordre = np.array([
        "bosses", # neteja
        "llet", "cocacola", "suc", # begudes
        "tortilla", "sazonador", "nachos", # mexica
        "ruffles", "triblis", "pipes", "olives", "escopinyes", "cigrons", 
        "fesols", #conserves
        "cookies", "cereals", "xocolata", #berenar
        "pasta", "arros", "tomaca solis", "tomaca tr", "pesto", #pasta
        "formatge", "salsitxes", "mortadela", "cheddar", "feta", 
        "iogurt", "ous", "nata", # refrigerats
        "pollastre", "bistec", "hamburguesa", # carn
        "tomaca pera", "tomacons", "alberginia", "pebrot", "cogollos", "ceba", "patates", "dents all", 
        "platan", "poma", "llim", "bajoca", "carxofa", "cogombre", "coriandre", # fruita i verdura
        "guacamole", "allioli", "salmo", "gelat", # congelats
        "pa ", "kinder", # pa i caixa
    ])
    vals = np.zeros(len(llista))
    for i, item in enumerate(ordre):
        mask = llista.index.str.startswith(item)
        vals[mask] = i+1
    llista = llista.to_frame("quantitat")
    llista["ordre"] = vals
    llista["index"] = llista.index
    llista = llista.sort_values(by=["ordre", "index"])
    return llista[["quantitat"]]
    

def llista_compra(menu, data):
    llista = pd.Series(np.zeros(data.index.shape), data.index)
    for key, vals in menu.items():
        llista += data[vals].sum(axis="columns")
    llista = llista.to_frame("item")
    llista = llista.loc[~np.isclose(llista.item, 0)]
    return llista

In [2]:
apats = {
    "arros alberginia": {"arros": 140, "alberginia": 1, "tomaca solis": 1, "formatge pols": 1/3},
    "salmo": {"salmo (caixa)": 1, "dents all": 2, "mostassa": 1/7},
    "amanida cogombre": {"cogombre": 1, "iogurt pastoret": 1/3},
    "hamburguesa": {"hamburguesa": 2, "pa hamburguesa": 2, "ceba": 1.5, "mostassa": 1/7, "formatge talls": 2},
    "truita patata": {"patates": 4, "ceba": 1, "ous": 4},
    "pa amb tomaca": {"tomacons": 2, "pa ": 2/5},
    "amanida arros": {
        "arros": 140, "cogollos": 1, "tomaca pera": 1, "ceba": 1/4, "feta": 1/2, "tonyina": 2, "ous": 2
    },
    "pure bajoca": {"bajoca": 330, "ceba": 1/2, "patates": 1, "formatge pols": 1/6, "nata": 1/6},
    "bistec": {"bistec": 2},
    "llom porc": {"llom": 200},
    "amanida": {"cogollos": 2, "tomaca pera": 2, "ceba": 1/4, "feta": 1/2},
    "pollastre": {"pollastre": 1},
    "pasta pesto": {"pesto": 1/3, "pasta": 160, "formatge pols": 1/3},
    "wrap+hummus": {"tortilla wrap": 2, "cigrons": 1/2, "llimona": 1, "medallons curry": 1/2},
    "amanida fesols": {
        "fesols": 1/2, "cogollos": 2, "tomaca pera": 2, "ceba": 1/3, "feta": 1/2, "ous": 2, 
        "tonyina": 2, "pebrot": 1/3
    },
    "fajitas": {"pebrot": 1, "pollastre": 1, "sazonador": 1/2, "ceba": 1, "tortilla": 6, "cheddar": 2/3},
    "patata caliu": {"patates": 4, "ceba": 2, "allioli": 1/2, "bistec": 2},
    "carxofa micro": {"carxofa": 2},
    "burrito": {
        "chili con carne": 1, "guacamole": 1, "ceba": 1, "tomaca pera": 2, "coriandre": .1, 
        "arros": 120, "cogollos": 2, "cheddar": 2/3, "tortilla wrap": 4, "llima": 1/2
    },
    "pasta arrabiata": {
        "tomaca triturat": 1, "tomaca trossejat": 1, "dents all": 3, "cayena": 1, 
        "pasta": 180, "salsitxes": 1, "formatge pols": 1/3
    },
    "amanida tomaca": {"tomaca pera": 3, "ceba": 1/4, "feta": 1/2},
    "suc tomaca": {"suc tomaca": 2/5},
    "pavo pimento": {"pavo pimento": 1/2},
    "curry": {"pollastre": 2, "pebrot": 1, "ceba": 1/2, "llet coco": 1, "llima": 1/2, "arros basmati": 170},
}
df = pd.DataFrame(apats).fillna(0)

In [3]:
menu = {
    "Dimarts 28": ["wrap+hummus", "amanida", "pure bajoca", "bistec"],
    "Dimecres 29": ["amanida cogombre", "salmo", "arros alberginia", "amanida fesols"],
    "Dijous 30": ["pasta pesto", "amanida", "pollastre", "amanida tomaca", "pavo pimento", "arros alberginia"],
    "Divendres 1": ["suc tomaca", "curry", "pure bajoca", "llom porc"],
    "Dissabte 2": ["amanida cogombre", "pavo pimento", "arros alberginia", "wrap+hummus", "amanida"],
    "Diumenge 3": [
        "amanida cogombre", "pasta pesto", "llom porc", "amanida tomaca", "truita patata", "pa amb tomaca"
    ],
    "Dilluns 4": ["amanida fesols"],
}

In [4]:
llista_manual = """
,item
cocacola, 1
llet, 2
llet sense lactosa, 2
cereals xocolata llet, 1
pipes, 3
mortadela, 2
formatge talls, 3
escopinyes, 1
olives, 1
xocolata negra amarga, 1
fairy, 1
gel, 1
gamuzas, 1
sifon, 1
cereals boles, 1
guants, 1
"""
llista_manual = pd.read_csv(
    StringIO(llista_manual), index_col=0
)

In [5]:
llista = llista_manual.merge(
    llista_compra(menu, df), 
    how="outer", 
    left_index=True, 
    right_index=True
).sum("columns", skipna=True)
llista = ordenar_llista(llista)
llista

,quantitat
cereals boles,1.000000
cereals xocolata llet,1.000000
fairy,1.000000
gamuzas,1.000000
gel,1.000000
guants,1.000000
llom,400.000000
medallons curry,1.000000
mostassa,0.142857
pavo pimento,1.000000
